In [1]:
print('hello')
import os
from sys import executable as exec
print('current working directory: ', os.getcwd())
print('current conda environment: ', exec)

#--notebook-dir=/home/spatial01/RND/adversarial-detector/adversarial-detector-xai

hello
current working directory:  /home/spatial01/RND/adversarial-detector/adversarial-detector-xai
current conda environment:  /home/spatial01/anaconda3/envs/netslab_lime_defense/bin/python


In [2]:
import DeepMIMO
import os
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from tqdm import tqdm
from scipy.io import loadmat, savemat 
import glob
import re 
import datetime as dt

# %% Machine Learning
import keras
import keras.models as models
from keras.layers import Dense,Dropout

# Adversarial attack libraries
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from art.estimators.classification import KerasClassifier
from myutils import evasion_attacks, create_dir_if_not_exist

plt.rcParams['figure.figsize'] = [12, 8] # Set default plot size

2025-04-14 18:56:32.879755: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-14 18:56:32.884249: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-14 18:56:32.947037: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 18:56:32.947073: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 18:56:32.947111: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [3]:
def beamforming_codebook(ant_shape = np.array([1, 32, 1]), oversampling_rate = np.array([1, 1, 1]), kd = 0.5):
    
    kd = 2 * np.pi * kd
    codebook_size = ant_shape * oversampling_rate
    
    vecs = []
    for dim in range(3):
        ind = np.arange(ant_shape[dim]).reshape((-1, 1))
        codebook_ang = np.linspace(0, np.pi, codebook_size[dim], endpoint = False).reshape((1, -1))                                                                                                     
        vec = np.sqrt(1./ant_shape[dim]) * np.exp(-1j * kd * ind * np.cos(codebook_ang))
        vecs.append(vec)
        
    F = np.kron(vecs[2], np.kron(vecs[1], vecs[0]))
    
    return F

# Generating the Dataset

In [4]:

##%% # Generate the dataset
# # Load and print the default parameters
parameters = DeepMIMO.default_params()


# # Change parameters for the setup
# Scenario O1_60 extracted at the dataset_folder
parameters['scenario'] = 'O1_60'
parameters['dataset_folder'] = os.path.abspath("assets/scenarios") # Set DeepMIMO dataset folder that has O1_60

parameters['num_paths'] = 5

# User rows 1000 to 1300
parameters['user_row_first'] = 1000
parameters['user_row_last'] = 1300

# Activate the BSs 3, 4, 5, 6
parameters['active_BS'] = np.array([ 3, 4, 5, 6]) 

parameters['OFDM']['bandwidth'] = 0.5 # 50 MHz
parameters['OFDM']['subcarriers'] = 1024 # OFDM with 512 subcarriers
parameters['OFDM']['subcarriers_limit'] = 64 # Keep only first 64 subcarriers

parameters['enable_BS2BS'] = False

parameters['ue_antenna']['shape'] = np.array([1, 1, 1]) # Single antenna
parameters['bs_antenna']['shape'] = np.array([1, 32, 8]) # UPA (2D matrix shaped antenna) of 256 elements (32 x 8)
parameters['bs_antenna']['radiation_pattern'] = 'halfwave-dipole'
parameters['ue_antenna']['radiation_pattern'] = 'halfwave-dipole'

pprint(parameters, sort_dicts = False)

{'dataset_folder': '/home/spatial01/RND/adversarial-detector/adversarial-detector-xai/assets/scenarios',
 'scenario': 'O1_60',
 'dynamic_settings': {'first_scene': 1, 'last_scene': 1},
 'num_paths': 5,
 'active_BS': array([3, 4, 5, 6]),
 'user_row_first': 1000,
 'user_row_last': 1300,
 'row_subsampling': 1,
 'user_subsampling': 1,
 'bs_antenna': {'shape': array([ 1, 32,  8]),
                'spacing': 0.5,
                'radiation_pattern': 'halfwave-dipole'},
 'ue_antenna': {'shape': array([1, 1, 1]),
                'spacing': 0.5,
                'radiation_pattern': 'halfwave-dipole'},
 'enable_BS2BS': False,
 'OFDM_channels': 1,
 'OFDM': {'subcarriers': 1024,
          'subcarriers_limit': 64,
          'subcarriers_sampling': 1,
          'bandwidth': 0.5,
          'RX_filter': 0}}


In [5]:
# %%script --false
# Takes around 3 mins to generate the dataset
dataset = DeepMIMO.generate_data(parameters)


Basestation 3

UE-BS Channels


Generating channels: 100%|██████████| 54481/54481 [00:34<00:00, 1591.38it/s]



Basestation 4

UE-BS Channels


Generating channels: 100%|██████████| 54481/54481 [00:32<00:00, 1683.74it/s]



Basestation 5

UE-BS Channels


Reading ray-tracing:  41%|████▏     | 97175/235300 [00:02<00:03, 38763.28it/s]


KeyboardInterrupt: 

In [12]:
##%% Parameters and Codebook
F = beamforming_codebook(ant_shape = parameters['bs_antenna'][0]['shape'], oversampling_rate = np.array([1, 2, 1]), kd = parameters['bs_antenna'][0]['spacing'])

num_OFDM = int(parameters['OFDM']['subcarriers_limit']/parameters['OFDM']['subcarriers_sampling'])
num_beams = F.shape[1]
num_bs = len(parameters['active_BS'])
num_ue = len(parameters['active_UE'])

NF = 5             # Noise figure at the base station
Process_Gain = 10  # Channel estimation processing gain
BW = parameters['OFDM']['bandwidth'] * 1e9 # System bandwidth in Hz
noise_power_dB = -204 + 10*np.log10(BW/parameters['OFDM']['subcarriers']) + NF - Process_Gain; # Noise power in dB
noise_power = 10**(.1*(noise_power_dB)); # Noise power


# Machine Learning

## Dataset preparation

In [ ]:
##%% DL Input-Output
input_norm = np.zeros((num_bs, num_ue, num_OFDM), dtype=complex)
max_rates = np.zeros((num_bs, num_ue, num_beams))
for bs_idx in tqdm(range(num_bs), desc='Neural Network Input-Output Generation-BS', position=0, leave=True):
    for ue_idx in tqdm(range(num_ue), desc='Neural Network Input-Output Generation-BS-%i'%bs_idx, position=0, leave=True):
        ch = dataset[bs_idx]['user']['channel'][ue_idx].squeeze()
        ch = ch + np.sqrt(noise_power) * (np.random.randn(*(ch.shape)) + 1j * np.random.randn(*(ch.shape)))
        input_norm[bs_idx, ue_idx, :] = ch[0, :]
        max_rates[bs_idx, ue_idx, :] = np.sum(np.log2(1 + np.abs(ch.T.conj() @ F)**2),  axis = 0)/num_OFDM
        
# Input reshape - normalize
input_norm = np.transpose(input_norm, axes=[1, 0, 2])
input_norm = input_norm.reshape((num_ue, -1))
input_norm /=  np.amax(np.abs(input_norm))

# Output reshape - normalize
max_rates_norm_factor = np.amax(max_rates, axis=2, keepdims=True)
max_rates_norm_factor[max_rates_norm_factor== 0] = 1 # Do not normalize if all zeros
max_rates /= max_rates_norm_factor
max_rates = np.transpose(max_rates, axes=[1, 0, 2])
max_rates = max_rates.reshape((num_ue, -1))

# Save the dataset
data_dir = create_dir_if_not_exist(os.path.join(os.getcwd(), "data/DeepMIMO_data/"))

# create_dir_if_not_exist(os.path.join(data_dir, "DLCB_dataset_"+dt.datetime.now().strftime("%d-%m-%y")))

## if not os.path.exists(os.path.join(data_dir, "DLCB_dataset_"+dt.datetime.now().strftime("%d-%m-0%y"))):
##     os.makedirs(os.path.join(data_dir, "DLCB_dataset_"+dt.datetime.now().strftime("%d-%m-%y")))

new_dataset_dir = create_dir_if_not_exist(os.path.join(data_dir, "DLCB_dataset_"+dt.datetime.now().strftime("%d-%m-%y")))

savemat(os.path.join(new_dataset_dir, 'DLCB_input.mat'), {'DL_input': input_norm})
savemat(os.path.join(new_dataset_dir,'DLCB_output.mat'), {'DL_output': max_rates})

input_data_path = os.path.join(new_dataset_dir, 'DLCB_input.mat')
output_data_path = os.path.join(new_dataset_dir,'DLCB_output.mat')


Neural Network Input-Output Generation-BS-2:  89%|████████▉ | 48760/54481 [02:17<00:22, 250.79it/s]

## Training and Testing

In [8]:
# Model training function
def train(In_train, Out_train, In_test, Out_test,
          epochs, batch_size,dr,
          num_hidden_layers, nodes_per_layer,
          loss_fn,n_BS,n_beams):
    
    in_shp = list(In_train.shape[1:])

    AP_models = []
    for bs_idx in range(n_BS):
        idx_str = 'BS%i' % bs_idx
        idx = bs_idx*n_beams
        
        model = models.Sequential()
        model.add(Dense(nodes_per_layer, activation='relu', kernel_initializer='he_normal', input_shape=in_shp))
        model.add(Dropout(dr))
        for h in range(num_hidden_layers):
            model.add(Dense(nodes_per_layer, activation='relu', kernel_initializer='he_normal'))
            model.add(Dropout(dr))
        
        model.add(Dense(n_beams, activation='relu', kernel_initializer='he_normal',
                  name="dense" + idx_str + "o"))
        model.compile(loss=loss_fn, optimizer='adam')
        model.summary()
    
        model.fit(In_train,
                    Out_train[:, idx:idx + n_beams],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=2,
                    validation_data=(In_test, Out_test[:,idx:idx + n_beams]),
                    callbacks = [
                        #keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
                        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
                    ])
        AP_models.append(model)
        
    return AP_models

In [9]:
# Reading input and output sets generated from MATLAB
In_set_file=loadmat(input_data_path)
Out_set_file=loadmat(output_data_path)

In_set=In_set_file['DL_input']
Out_set=Out_set_file['DL_output']

# Parameter initialization
num_user_tot=In_set.shape[0]
n_DL_size=[0.001, .05, .1, .15, .2, .25, .3, .35, .4, .45, .5, .55, .6, .65, .7, .75, .8, .85, .9, .95, 1.0]
num_tot_TX=4
num_beams=512

### Training Loop

In [ ]:
# %%script --false
# Define it in this cell because it changes when the inside the for loop
count=0

results_dir = create_dir_if_not_exist(os.path.join(os.getcwd(), "results/DeepMIMO_data_"+dt.datetime.now().strftime("%d-%m-%y")+"/"))
output_model_dir = create_dir_if_not_exist(os.path.join(results_dir, "Models"))
adv_data_dir = create_dir_if_not_exist(os.path.join(results_dir, 'AdvData'))

for DL_size_ratio in n_DL_size:
    
    print (DL_size_ratio)
    count=count+1
    DL_size=int(num_user_tot*DL_size_ratio)
    
    np.random.seed(2016)
    n_examples = DL_size
    num_train  = int(DL_size * 0.8)
    num_test   = int(num_user_tot*.2)
    
    train_index = np.random.choice(range(0,num_user_tot), size=num_train, replace=False)
    rem_index = set(range(0,num_user_tot))-set(train_index)
    test_index= list(set(np.random.choice(list(rem_index), size=num_test, replace=False)))
    
    In_train = In_set[train_index]
    In_test =  In_set[test_index] 
        
    Out_train = Out_set[train_index]
    Out_test = Out_set[test_index]
    
    
    # Learning model parameters
    epochs = 10     
    batch_size = 100  
    dr = 0.05                  # dropout rate  
    num_hidden_layers=4
    nodes_per_layer=In_train.shape[1]
    loss_fn='binary_crossentropy'
    
    # Model training
    AP_models = train(In_train, Out_train, In_test, Out_test,
                                          epochs, batch_size,dr,
                                          num_hidden_layers, nodes_per_layer,
                                          loss_fn, num_tot_TX, num_beams)
    
    # Save the benign models
    for n_BS, model in enumerate(AP_models):
        create_dir_if_not_exist(os.path.join(output_model_dir, f"{DL_size_ratio}"))
        model.save(os.path.join(output_model_dir, f"{DL_size_ratio}/model_{n_BS}.SavedModel"), save_format='tf')

    # Model running/testing
    DL_Result={}
    for idx in range(0,num_tot_TX,1): 
        # adversarial data injection
        art_model = KerasClassifier(model=AP_models[idx], use_logits=False)
        attack_type = "fgsm"
        verbose = 2
        print("Generating {} attack data".format(attack_type))
        fgsm_wrapped_model, fgsm_data = evasion_attacks.generate_evasion_data(wrapped_model=art_model
                                                                            , attack_type=attack_type
                                                                            , X_test=In_set
                                                                            , verbose=verbose
                                                                            , eps=0.4
                                                                            , confidence=0.9
                                                                            , nb_parallel=64
                                                                             )
        fgsm_data_test = fgsm_data[test_index]
                
        # Save the adversarial data as .mat file
        create_dir_if_not_exist(os.path.join(adv_data_dir, f"{DL_size_ratio}"))
        print(f"Saving adversarial data for {DL_size_ratio} and {idx}")
        savemat(os.path.join(adv_data_dir, f"{DL_size_ratio}/adv_data_test_{idx}.mat"), {'adv_data_test': fgsm_data_test})
        savemat(os.path.join(adv_data_dir, f"{DL_size_ratio}/adv_data_full_{idx}.mat"), {'adv_data_full': fgsm_data})
        
        beams_predicted=AP_models[idx].predict( fgsm_data_test, batch_size=10, verbose=0)
        DL_Result['TX'+str(idx+1)+'Pred_Beams']=beams_predicted
        DL_Result['TX'+str(idx+1)+'Opt_Beams']=Out_test[:,idx*num_beams:(idx+1)*num_beams]

    DL_Result['user_index']=test_index
    
    DLCB_code_output_dir = create_dir_if_not_exist(os.path.join(results_dir, 'DLCB_code_output'))
    # if not os.path.exists(os.path.join(results_dir, 'DLCB_code_output')):
    #                       os.makedirs(os.path.join(results_dir, 'DLCB_code_output'))
    savemat(os.path.join(DLCB_code_output_dir, 'DL_Result_')+str(count)+'.mat', DL_Result)

# Read Results

In [ ]:
##%% Read Results
file_list = sorted(glob.glob(os.path.join(results_dir, 'DLCB_code_output/DL_Result*')), key=lambda x: int(re.findall(r'\d+', x)[0]))
num_files = len(file_list)

user_index = []
pred_beams = []
opt_beams = []
for file in tqdm(file_list, desc='Reading DL results'):
    matfile = loadmat(file)
    l1 = []
    l2 = []
    for idx in range(num_bs):
        l1.append(matfile['TX'+str(idx+1)+'Pred_Beams'])
        l2.append(matfile['TX'+str(idx+1)+'Opt_Beams'])
        
    pred_beams.append(l1)
    opt_beams.append(l2)
    user_index.append(matfile['user_index'])

In [ ]:
user_index[file_idx].flatten()

In [ ]:


Pn = -204 + 10*np.log10(BW) # Noise power in dB
SNR = 10**(.1*(0-Pn))

ach_rate_DL = np.zeros(num_files)
ach_rate_opt = np.zeros(num_files)

eff_rate = np.zeros(num_files)
opt_rate = np.zeros(num_files)
for file_idx in tqdm(np.arange(num_files), desc = 'Calculating results'):
    user_index_file = user_index[file_idx].flatten()
    for ue_idx in range(len(user_index_file)):
        eff_ch = []
        opt_ch = []
        for bs_idx in range(num_bs):
            if file_idx == 0: # Random BF - 0 Samples
                pred_beam_idx = np.random.randint(num_beams)
            else:
                pred_beam_idx = np.argmax(pred_beams[file_idx][bs_idx][ue_idx])
            opt_beam_idx = np.argmax(opt_beams[file_idx][bs_idx][ue_idx])
            ch_single_bs = dataset[bs_idx]['user']['channel'][user_index_file[ue_idx]].squeeze()
            eff_ch_single_pred = ch_single_bs.T.conj() @ F[:, pred_beam_idx]
            opt_ch_single_pred = ch_single_bs.T.conj() @ F[:, opt_beam_idx]
            eff_ch.append(eff_ch_single_pred)
            opt_ch.append(opt_ch_single_pred)
        eff_ch = np.array(eff_ch)
        opt_ch = np.array(opt_ch)
        eff_rate[file_idx] += np.sum(np.log2(1 + SNR * np.abs(np.diag(eff_ch.conj().T @ eff_ch))))
        opt_rate[file_idx] += np.sum(np.log2(1 + SNR * np.abs(np.diag(opt_ch.conj().T @ opt_ch))))
    eff_rate[file_idx] /= len(user_index_file)*num_OFDM
    opt_rate[file_idx] /= len(user_index_file)*num_OFDM


# % Eff achievable rate calculations
theta_user=(102/parameters['bs_antenna'][0]['shape'][1])*np.pi/180
alpha=60*np.pi/180
distance_user=10
Tc_const=(distance_user*theta_user)/(2*np.sin(alpha)) # ms
Tt=10*1e-6; # ms

v_mph=50
v=v_mph*1000*1.6/3600 # m/s
Tc=Tc_const/v

overhead_opt=1-(num_beams*Tt)/Tc # overhead of beam training
overhead_DL=1-Tt/Tc # overhead of proposed DL method

##%% Plotting the figure
DL_size_array=np.arange(0, 2.5*num_files, 2.5);


# if not os.path.exists(os.path.join(results_dir, 'Figs')):
#     os.makedirs(os.path.join(results_dir, 'Figs'))

results_figs_dir = create_dir_if_not_exist(os.path.join(results_dir, 'Figs'))

fig, ax = plt.subplots()
plt.plot(DL_size_array, opt_rate, '--k', label = 'Genie-aided Coordinated Beamforming')
plt.plot(DL_size_array, eff_rate*overhead_DL, '-bo', label = 'Deep Learning Coordinated Beamforming')
plt.plot(DL_size_array, opt_rate*overhead_opt, '-rs', label = 'Baseline Coordinated Beamforming')
plt.ylim([0, 6])
plt.minorticks_on()
plt.grid()
plt.xlabel('Deep Learning Dataset Size (Thousand Samples)')
plt.ylabel('Achievable Rate (bps/Hz)')
plt.legend()
current_time = dt.datetime.now().strftime("%d-%m-%y-%H%M%S")
plt.savefig(os.path.join(results_figs_dir, f'result{current_time}.png'))

In [ ]:
print("EOF - Code has finished executing !")